In [1]:
import xarray as xr
import numpy as np
import os

from pcv.cfg import IPCC_ACRONYM

In [23]:
xtreme = "high"
vegetation_type = "forest"
ds_path = "/data/compoundx/anand/PCV/data/"

In [24]:
region_list = list(IPCC_ACRONYM.keys())

In [25]:
for region in region_list:
    data = xr.open_dataset(f"/data/compoundx/anand/PCV/data/{vegetation_type}_data/{xtreme}/{vegetation_type}_{region}_v3.nc")
    data = data.sortby(data.longitude).sortby(data.latitude).sortby(data.time)
    data_flat = data.stack(point =("latitude", "longitude", "time"))
    data_flat = data_flat.dropna(dim="point", how="any")
    var_list = list(data_flat.keys())

    data_list = []
    for i, var in enumerate(var_list):
        data_i = data_flat[var]
        if i < len(var_list)-1:
            assert (data_i.time == data_flat[var_list[i+1]].dropna(dim = "point", how="all").time).all().values == True

        if i==0:
            data_list.append(data_i.time.values)
        data_list.append(data_i.values)

    data_arr = np.vstack(data_list).T
    data_arr[~np.isnan(data_arr).any(axis=1)]

    header_names = [each_header.replace(f"_{vegetation_type}", "") for each_header in var_list]

    header = "year\t"
    header = header + ("\t".join(header_names))
    np.savetxt(f"{ds_path}{vegetation_type}_data/{xtreme}/"+f"{vegetation_type}_{xtreme}_{region}_v3.csv", data_arr, header=header, fmt='%1.3f', delimiter="\t")
    print(region)


C.North-America
E.Asia
E.C.Asia
E.Europe
E.North-America
E.Siberia
Mediterranean
N.Central-America
N.E.North-America
N.Europe
N.W.North-America
Russian-Arctic
Russian-Far-East
S.Asia
Tibetan-Plateau
W.C.Asia
W.North-America
W.Siberia
West&Central-Europe
